###  AT KAMIS _ 일별 품목별 도·소매가격정보 API 

In [1]:
import requests
import pprint
import json
import pandas as pd
from pandas.io.json import json_normalize
import xmltodict
from bs4 import BeautifulSoup

### API 이용 데이터 파싱
이렇게도 할 수 있지만, 이번 분석에서는 공공데이터에 연도별로 올라와 있는 csv 파일 이용해서 분석함.

In [2]:
key = '6afbdb611f88b65c19d0ac313cc7eac05d5fdb390ecd2de82fa14566c2fb03e4'
apiUrl = 'Grid_20161221000000000429_1'
startIndex = 1
endIndex = 1000

# 범위 시작 및 종료 날짜
start_date = '2023-05-01'
end_date = '2023-05-10'

# 지정된 범위의 날짜를 포함하는 시리즈 생성
date_series = pd.date_range(start=start_date, end=end_date, freq='D')

# 날짜를 원하는 숫자 형식으로 변환
numeric_dates = date_series.strftime('%Y%m%d').astype(int)

# 숫자 형식의 날짜를 포함하는 데이터프레임 생성
date_df = pd.DataFrame(numeric_dates, columns=['가격등록일자'])
# 숫자 형식의 날짜를 포함하는 데이터프레임 리스트로 변경
dateList = date_df['가격등록일자'].tolist()

# 품종코드 리스트
pumjongCd_list = ['110301']

pummok_df = pd.DataFrame()

# 지정한 기간동안 for문 반복
for date in dateList:
    for pumjongCd in pumjongCd_list:
        url = f'http://211.237.50.150:7080/openapi/{key}/xml/{apiUrl}/{startIndex}/{endIndex}?AUCNG_DE={date}&SPCIES_CD={pumjongCd}'
        response = requests.get(url)
        xml_data = xmltodict.parse(response.content)
        data = xml_data[apiUrl]
        print(date)
        print("날짜별 데이터 수: "+data['totalCnt'])
        data_count = int(data['totalCnt'])
        if data_count > 0:
            data_list = xml_data[apiUrl]['row']
            daily_df = pd.DataFrame(data_list)
            pummok_df = pd.concat([pummok_df, daily_df])
        else:
            # totalCnt가 0인 경우 빈 데이터프레임을 생성하고 NaN 값으로 채운 다음 데이터를 합침
            empty_daily_df = pd.DataFrame(columns=pummok_df.columns)
            empty_daily_df.loc[0,:] = pd.NA # 모든 값이 결측값인 행을 추가
            empty_daily_df['AUCNG_DE'] = date
            pummok_df = pd.concat([pummok_df, empty_daily_df])

pummok_df = pummok_df.drop([pummok_df.columns[0]],axis=1)
pummok_df.columns = ['datadate', '도매시장명', '도매시장코드', '법인명', '법인코드','원표번호','일련번호','경매시간','품목명','품목코드','품종명','품종코드','경락가','거래단량','규격','규격코드','등급','등급코드','산지코드','산지명','거래량']
# pummok_df = pummok_df['datadate', '도매시장명', '도매시장코드', '법인명', '법인코드','품목명','품목코드','품종명','품종코드','경락가','거래단량','규격','규격코드','등급','등급코드','산지코드','산지명','거래량']


pummok_df.to_excel('20230616_경락_병합_test.xlsx',index=True)

20230501
날짜별 데이터 수: 142
20230502
날짜별 데이터 수: 192
20230503
날짜별 데이터 수: 153
20230504
날짜별 데이터 수: 109
20230505
날짜별 데이터 수: 141
20230506
날짜별 데이터 수: 43
20230507
날짜별 데이터 수: 0
20230508
날짜별 데이터 수: 158
20230509
날짜별 데이터 수: 172
20230510
날짜별 데이터 수: 132


### 농산물 도소매 가격_ 연도별 CSV file 이용
데이터 파싱

In [5]:
import pandas as pd
import os

In [6]:
# 연도별 엑셀 파일 명 리스트
years = list(range(2017,2024))
file_list = [f"raw_doso/TB_TAT_KAMIS_PRC_MK_TB-{year}.csv" for year in years]
file_list

['raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2017.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2018.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2019.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2020.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2021.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2022.csv',
 'raw_doso/TB_TAT_KAMIS_PRC_MK_TB-2023.csv']

In [8]:
# 범위 시작 및 종료 날짜
start_date = '2017-01-01'
end_date = '2023-05-10'

# 지정된 범위의 날짜를 포함하는 시리즈 생성
date_series = pd.date_range(start=start_date, end=end_date, freq='D')

# 날짜를 원하는 숫자 형식으로 변환
numeric_dates = date_series.strftime('%Y%m%d').astype(int)

# 숫자 형식의 날짜를 포함하는 데이터프레임 생성
date_df = pd.DataFrame(numeric_dates, columns=['가격등록일자'])

print(date_df)

        가격등록일자
0     20170101
1     20170102
2     20170103
3     20170104
4     20170105
...        ...
2316  20230506
2317  20230507
2318  20230508
2319  20230509
2320  20230510

[2321 rows x 1 columns]


In [14]:
# 저장할 df를 저장하는 딕셔너리
dfs = {}

# # df에서 농산물 항목 목록
# # 품목명
# products = ['방울토마토','고구마','감자','배추','양배추','시금치','건고추','토마토','무','깐마늘(국산)','양파','미나리','깻잎','파프리카','팽이버섯','새송이버섯','얼갈이배추','당근']
# 품종명
# products = ['청양고추','꽈리고추','청','다다기계통','가시계통','취청','애호박','쥬키니','대파','쪽파']
products = ['청양고추']

# 도매 소매 구분
channels = ['도매', '소매']

# 모든 연도별 파일 순회
for file_name in file_list:
    df = pd.read_csv(f"{file_name}", encoding='cp949')
    df.columns = ['가격등록일자','시장고유번호','시장코드','시장명','시도코드','시도명','시군구코드','시군구명','품목코드','품목명','품종코드','품종명','도소매조사구분코드','조사구분명','산물등급코드','산물등급명','품목가격','산지출하단위크기','산지출하단위명','도매출하단위크기','도매출하단위명','소매출하단위크기','소매출하단위명','친환경농산물출하단위크기','친환경농산물출하단위명','할인가격여부','ETL적재일시']
    df = df[['가격등록일자','시장코드','시장명','시도코드','시도명','품목코드','품목명','품종코드','품종명','조사구분명','산물등급코드','산물등급명','품목가격','도매출하단위크기','도매출하단위명','소매출하단위크기','소매출하단위명']]
    df['품종명'] = df['품종명'].astype(str)

    # 각 농산물별로 df 분할
    for product in products:
        product_df = df[df['품종명'] == product] # 해당 농산물에 대한 df

        for channel in channels:
            channel_df = product_df[product_df['조사구분명'] == channel]
            channel_df = channel_df[['가격등록일자','시장코드','시장명','시도코드','시도명','품목코드','품목명','품종코드','품종명','조사구분명','산물등급코드','산물등급명','품목가격',f'{channel}출하단위크기',f'{channel}출하단위명']]

            # 딕셔너리에 농산물 데이터 프레일 저장
            product_channel_key = f"{product}_{channel}"
            if product_channel_key not in dfs:
                dfs[product_channel_key] = channel_df
            else:
                # pd.concat 사용해 df 연결
                dfs[product_channel_key] = pd.concat([dfs[product_channel_key], channel_df])


/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_25214/4012928245.py:16: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{file_name}", encoding='cp949')
/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_25214/4012928245.py:16: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{file_name}", encoding='cp949')
/var/folders/7d/d1khjvv95fs1dx3zplr2sfzh0000gn/T/ipykernel_25214/4012928245.py:16: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{file_name}", encoding='cp949')


In [15]:
# 각 농산물별로 새로운 엑셀 파일 저장
output_folder = "output/doso_output"
for product_channel_key, product_channel_df in dfs.items():
    product_channel_df = pd.merge(date_df,product_channel_df, left_on='가격등록일자', right_on='가격등록일자', how='outer')
    # 필요하다면 인덱스를 리셋하여 저장
    output_df = product_channel_df.reset_index(drop=True)
    output_path = os.path.join(output_folder, f"{product_channel_key}_prices.xlsx")
    output_df.to_excel(output_path, index=False)